In [31]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [32]:
class contextual_bandit():
    
    def __init__(self):
        self.state = 0
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0, self.num_bandits) # returns random bandit per episode
        return self.state
        
    def pullArm(self, action):
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

In [33]:
class agent():
    
    def __init__(self, lr, s_size, a_size):
        self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in, s_size)
        output = slim.fully_connected(state_in_OH, a_size, biases_initializer=None, activation_fn=tf.nn.sigmoid,
                                     weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output,[-1])  #flatten
        self.chosen_action = tf.argmax(self.output, 0)
        
        self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [34]:
tf.reset_default_graph()

In [35]:
cBandit = contextual_bandit()

In [36]:
myAgent = agent(lr=0.001, s_size=cBandit.num_bandits, a_size=cBandit.num_actions)

In [43]:
weights = tf.trainable_variables()[0]
print(weights)

Tensor("fully_connected/weights/read:0", shape=(3, 4), dtype=float32)


In [38]:
total_episodes = 10000
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions]) #scoreboard for bandits
e = 0.1

In [39]:
init = tf.global_variables_initializer()

In [40]:
with tf.Session() as sess:
    sess.run(init)
    i=0
    while i<total_episodes:
        s=cBandit.getBandit() #get state/random bandit from the environment
        
        # choose random action or one from network
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action, feed_dict={myAgent.state_in:[s]})
            
        reward = cBandit.pullArm(action)
        
        #update the network
        _, ww = sess.run([myAgent.update, weights], feed_dict={myAgent.reward_holder:[reward],
                                                              myAgent.action_holder:[action],
                                                              myAgent.state_in:[s]})
        
        total_reward[s,action] += reward
        if i%500==0:
            print("Mean rewardfor each bandit"+str(np.mean(total_reward, axis=1)))
        i+=1
        
print(ww)

for i in range(cBandit.num_bandits):
    print("the agent thinks action "+str(np.argmax(ww[i])+1) + " for bandit "+str(i+1))

Mean rewardfor each bandit[-0.25  0.    0.  ]
Mean rewardfor each bandit[ 35.25  30.5   33.5 ]
Mean rewardfor each bandit[ 72.5   69.    69.25]
Mean rewardfor each bandit[ 108.25  108.25  103.25]
Mean rewardfor each bandit[ 150.75  144.5   136.  ]
Mean rewardfor each bandit[ 191.    181.5   171.75]
Mean rewardfor each bandit[ 229.25  216.    208.5 ]
Mean rewardfor each bandit[ 265.25  252.    242.5 ]
Mean rewardfor each bandit[ 300.25  289.5   278.  ]
Mean rewardfor each bandit[ 335.75  328.    315.  ]
Mean rewardfor each bandit[ 373.    366.5   349.75]
Mean rewardfor each bandit[ 413.75  401.25  385.75]
Mean rewardfor each bandit[ 452.25  436.    420.  ]
Mean rewardfor each bandit[ 495.75  472.5   449.5 ]
Mean rewardfor each bandit[ 533.25  513.25  481.25]
Mean rewardfor each bandit[ 569.75  553.    518.  ]
Mean rewardfor each bandit[ 605.5   591.25  550.5 ]
Mean rewardfor each bandit[ 640.75  633.25  587.25]
Mean rewardfor each bandit[ 682.75  670.5   620.5 ]
Mean rewardfor each band

In [30]:
with tf.Session() as s:
    print(s.run(weights))
    

FailedPreconditionError: Attempting to use uninitialized value fully_connected/weights
	 [[Node: fully_connected/weights/_0 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_4_fully_connected/weights", _device="/job:localhost/replica:0/task:0/gpu:0"](fully_connected/weights)]]
	 [[Node: fully_connected/weights/_1 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_4_fully_connected/weights", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]